In [24]:
## System
import re
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

## EDA
import numpy as np
import pandas as pd
import seaborn as sns
from pandas_profiling import ProfileReport
pd.set_option('display.max_columns', None)

In [2]:
#import classification (decision tree, random forest, logistic regression, xgboost, catboost)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
#from xgboost import XGBClassifier

In [63]:
##----------------------BASE FUNCTIONS FOR READING CSV AND FILTERING DATA

## IMPORT DATA FROM CSV FILE
def ImportData(team_file, type):
    folder = team_file + '/'
    maps_file = team_file + '_'+type+'_MAPS.csv'
    matches_file = team_file + '_'+type+'_MATCHES.csv'
    
    maps = pd.read_csv(CSV_PATH + folder + maps_file)
    matches = pd.read_csv(CSV_PATH + folder + matches_file)

    return maps, matches

## FILTER DATAFRAME BY GIVEN COLUMN ON GIVEN VALUE
def FilterCol(df, col, values):
    return df[df[col].isin(values)].reset_index(drop=True)

def BinaryTarget(variavel, quantil):
  bin = (variavel > variavel.quantile(q = quantil)).astype(int)
  return bin

##----------------------IMPORT + FILTERING DATA

## Aggregate base_geral by (Team, Match, Map)
def aggBase(base_geral):
    
    mean_agg_cols  = ['ACS', 'K',	'D',	'A',	'KD_DIFF',	'ADR',	'HS%', 'ECON']
    
    sum_agg_cols   = ['FK',	'FD', 'KPR',	'DPR',	'APR',	'FKPR'	,'FDPR',	
                        '2K'	,'3K',	'4K'	,'5K',	
                        '1v1','1v2',	'1v3'	,'1v4'	,'1v5' ,'PL',	'DE', 
                        'total_mult_kill', 'MKPR', 'total_clutch' ,'CPR'] + AGENTS
    
    first_agg_cols = ['Opp_Team_x', 'rounds_won'	,'rounds_lost', 'ct_rounds_won','ct_rounds_lost','t_rounds_won','t_rounds_lost' ,	
                        'total_rounds', 'win_rate', 'ct_wr' , 't_wr', 'RESULT']

    
    '''['Pistol_W','Pistol_P','Pistol_WR' ,'Eco_0_5_P','Eco_0_5_W','Eco_0_5_WR' ,
    'Semi_Eco_5_10_P', 'Semi_Eco_5_10_W','Semi_Eco_5_10_WR', 
    'Semi_Buy_10_20_P','Semi_Buy_10_20_W','Semi_Buy_10_20_WR',
    'Full_Buy_20_P'	,'Full_Buy_20_W','Full_Buy_20_WR']'''

    agg_first = base_geral.groupby(['Team_x','match_id','Map']).first()[first_agg_cols].reset_index()
    agg_sum = base_geral.groupby(['Team_x','match_id','Map']).sum()[sum_agg_cols].reset_index()
    agg_mean = base_geral.groupby(['Team_x','match_id','Map']).mean()[mean_agg_cols].reset_index().round(2)

    merge_agg = agg_mean.merge(agg_sum, how='inner', left_on=['Team_x','match_id','Map'],right_on=['Team_x','match_id','Map']).drop_duplicates()
    merge_agg = merge_agg.merge(agg_first, how='inner', left_on=['Team_x','match_id','Map'],right_on=['Team_x','match_id','Map']).drop_duplicates()

    #merge_agg = merge_agg.merge(economy_maps, how='inner', left_on=['match_id','Map','Team_x'],right_on=['match_id','Map','Team']).drop_duplicates()

    return merge_agg
    
## Get Map data for all teams
def AllTeamsMaps(files, type, team_col, filter_teams):
    concat = pd.concat([ImportData(team_file, type)[0] for team_file in files])
    #filter = FilterCol(concat, team_col, filter_teams)
    filter = concat

    filter[team_col][(filter[team_col] == 'NOOR') | (filter[team_col] == 'NOORG2.0')] = 'paiN'
    filter[team_col][filter[team_col] == 'Mix'] = 'IMP'
    filter[team_col][(filter[team_col] == 'Ingaming') | (filter[team_col] == 'NMDM')] = 'Inga'
    filter[team_col][filter[team_col] == 'FF'] = 'VORA'
    filter[team_col][(filter[team_col] == 'HAVA') | (filter[team_col] == 'HL')] = 'HAVAN'
    filter['Opp_Team'][(filter['Opp_Team'] == 'HAVA') | (filter['Opp_Team'] == 'HL')] = 'HAVAN'
    filter[team_col][filter[team_col] == 'FURI'] = 'FURIA'
    filter[team_col][(filter[team_col] == 'S5') | (filter[team_col] == 'Shar')] = 'Sharks'
    
    
    if type == 'overview': ExtraInfoMaps(filter)
    if type == 'performance':
        # total mult kill + mult kills per round
        filter['total_mult_kill'] = filter.loc[: ,['2K','3K','4K','5K']].sum(axis=1)	

        # total clutches + clutches per round
        filter['total_clutch'] = filter.loc[: ,['1v1','1v2','1v3','1v4','1v5']].sum(axis=1)
    return filter

## Get Match data for all teams
def AllTeamsMatches(files, type, team_col, filter_value):
    concat = pd.concat([ImportData(team_file, type)[1] for team_file in files])
    filter = FilterCol(concat, team_col, filter_value)
    
    filter[team_col][(filter[team_col] == 'NOOR') | (filter[team_col] == 'NOORG2.0')] = 'paiN'
    filter[team_col][filter[team_col] == 'Mix'] = 'IMP'
    filter[team_col][(filter[team_col] == 'Ingaming') | (filter[team_col] == 'NMDM')] = 'Inga'
    filter[team_col][filter[team_col] == 'FF'] = 'VORA'
    filter[team_col][filter[team_col] == 'HAVA'] = 'HAVAN'
    filter[team_col][filter[team_col] == 'FURI'] = 'FURIA'
    filter[team_col][(filter[team_col] == 'S5') | (filter[team_col] == 'Shar')] = 'Sharks'
    
    if type == 'overview': ExtraInfoMatches(filter)
    return filter

## ADD EXTRA INFO TO MATCHES DATAFRAME -- OVERVIEW
def ExtraInfoMatches(df):
    # KDA + FK/FD per map
    df['KPM'] = round(df['K'] / df['Num_maps'],2)
    df['DPM'] = round(df['D'] / df['Num_maps'],2)
    df['APM'] = round(df['A'] / df['Num_maps'],2)
    df['FKPM'] = round(df['FK'] / df['Num_maps'],2)
    df['FDPM'] = round(df['FD'] / df['Num_maps'],2)

## CLUTCH TOTAL, CLUTCH PER ROUND, MULT KILL TOTAL, MULT KILL PER ROUND

## ADD EXTRA INFO TO MAPS DATAFRAME -- OVERVIEW
def ExtraInfoMaps(df):

    # Total Rounds
    df['total_rounds'] = df['rounds_won'] + df['rounds_lost']

    # Round Win Rate
    df['win_rate'] = round(df['rounds_won'] / df['total_rounds'] * 100, 2)

    # KDA + FK/FD per Round
    df['KPR'] = round(df['K'] / df['total_rounds'],2)
    df['DPR'] = round(df['D'] / df['total_rounds'],2)
    df['APR'] = round(df['A'] / df['total_rounds'],2)
    df['FKPR'] = round(df['FK'] / df['total_rounds'],2)
    df['FKWR'] = round(100 * df['FK'] / (df['FK'] + df['FD']), 2)
    df['FDPR'] = round(df['FD'] / df['total_rounds'],2)

    # ATK/DEF Win rate -- ct_rounds_won	ct_rounds_lost	t_rounds_won	t_rounds_lost
    df['ct_wr'] = round(100 * df['ct_rounds_won'] / (df['ct_rounds_won'] + df['ct_rounds_lost']), 2)
    df['t_wr'] = round(100 * df['t_rounds_won'] / (df['t_rounds_won'] + df['t_rounds_lost']), 2)


    # Won/Lost Map 
    df['RESULT'] = ['W' if x == True
                    else 'L'
                    for x in df['rounds_won'] > df['rounds_lost']]

## One Hot Encoding Categorical Cols
def GetDummies(df, cat_cols):

    for col in cat_cols:
        one_hot = pd.get_dummies(df[col])
        #df.drop(col, axis=1, inplace=True)
        df = pd.concat([df, one_hot], axis=1)
    return df


In [64]:
ALL_FILES = ['gamelanders', 'pain', 'vorax', 'havan', 'imperial', 'ingaming', 'vikings', 'black_dragons', 'furia', 'rise', 'slick', 'sharks']
ALL_TEAMS = ['GL', 'IMP','Mix', 'HAVAN', 'HAVA', 'Inga', 'Ingaming', 'NMDM', 'VKS', 'FF', 'VORA', 'BD', 'paiN','NOOR', 'NOORG2.0', 
                'S5', 'Sharks', 'Shar', 'FURI', 'FURIA', 'Rise', 'SLK']

MAPS = ['ASCENT', 'BIND', 'HAVEN', 'ICEBOX', 'SPLIT']

DUELIST = ['jett', 'raze', 'phoenix','reyna', 'yoru']
CONTROLLER = ['omen', 'brimstone', 'viper', 'astra'] 
SENTINEL = ['cypher', 'sage', 'killjoy'] 
INITIATOR = ['breach', 'sova', 'skye']
AGENTS = DUELIST + CONTROLLER + SENTINEL + INITIATOR

CSV_PATH = '../data/csv/'
EXCEL_PATH = '../data/excel/'

In [67]:
performance_maps.head()

,Player,2K,3K,4K,5K,1v1,1v2,1v3,1v4,1v5,ECON,PL,DE,match_id,Team,Opp_Team,Map,Patch,total_mult_kill,total_clutch
0,Nyang,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,30,0,1,6797,GL,IMP,SPLIT,1.12,2.0,1.0
1,fznnn,3,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,31,0,0,6797,GL,IMP,SPLIT,1.12,3.0,1.0
2,JhoW,3,0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,38,3,0,6797,GL,IMP,SPLIT,1.12,3.0,2.0
3,Jonn,6,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,115,0,0,6797,GL,IMP,SPLIT,1.12,8.0,1.0
4,mwzera,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74,0,0,6797,GL,IMP,SPLIT,1.12,4.0,0.0


In [68]:
overview_maps = AllTeamsMaps(ALL_FILES, 'overview', 'Team', ALL_TEAMS)
performance_maps = AllTeamsMaps(ALL_FILES, 'performance', 'Team', ALL_TEAMS)
#economy_maps = AllTeamsMaps(ALL_FILES, 'economy', 'Team', ALL_TEAMS)

In [69]:
overview_maps.drop('Patch',axis=1, inplace=True)
performance_maps.drop('Patch',axis=1, inplace=True)
base_geral = overview_maps.merge(performance_maps, how='inner', left_on=['match_id','Map','Player'],right_on=['match_id','Map','Player']).drop_duplicates()
#base_geral = merge_over_perf.merge(economy_maps, how='inner', left_on=['match_id','Map','Team_x'],right_on=['match_id','Map','Team']).drop_duplicates()
base_geral = base_geral.replace(np.nan, 0)
base_geral = GetDummies(base_geral, ['Agents'])
base_geral['MKPR'] = round(base_geral['total_mult_kill'] / base_geral['total_rounds'], 2)
base_geral['CPR'] = round(base_geral['total_clutch'] / base_geral['total_rounds'], 2)

In [70]:
base_agg = aggBase(base_geral)
base_agg = base_agg.replace(np.nan, 0)
base_agg[AGENTS] = base_agg[AGENTS].replace(2, 1)
print(base_agg.shape) 

(1068, 60)


In [9]:
base_agg.sort_values('match_id')

,Team_x,match_id,Map,ACS,K,D,A,KD_DIFF,ADR,HS%,ECON,FK,FD,KPR,DPR,APR,FKPR,FDPR,2K,3K,4K,5K,1v1,1v2,1v3,1v4,1v5,PL,DE,total_mult_kill,MKPR,total_clutch,CPR,jett,raze,phoenix,reyna,yoru,omen,brimstone,viper,astra,cypher,sage,killjoy,breach,sova,skye,Opp_Team_x,rounds_won,rounds_lost,ct_rounds_won,ct_rounds_lost,t_rounds_won,t_rounds_lost,Patch_x,total_rounds,win_rate,ct_wr,t_wr,RESULT
313,HAVAN,2443,BIND,147.2,10.0,14.2,3.4,-4.2,100.60,22.6,41.8,7,13,2.50,3.55,0.85,0.35,0.65,6,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,1,9.0,0.45,0.0,0.00,0,1,1,1,0,1,0,0,0,0,0,0,0,1,0,FF,7,13,4,8,3,5,1.07,20,35.00,33.33,37.50,L
816,VORA,2443,BIND,201.6,14.2,10.0,5.6,4.2,124.10,25.2,50.0,13,7,3.55,2.50,1.40,0.65,0.35,8,6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,9,0,15.0,0.75,0.0,0.00,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,HAVAN,13,7,5,3,8,4,1.07,20,65.00,62.50,66.67,W
1072,tiob,2453,ASCENT,182.4,14.6,18.0,6.4,-3.4,122.52,18.0,45.0,11,12,3.17,3.92,1.38,0.49,0.52,11,3,0.0,0.0,2.0,0.0,0.0,0.0,0.0,5,2,14.0,0.62,2.0,0.08,1,0,0,0,0,1,0,0,0,1,0,0,1,1,0,HAVAN,10,13,7,5,3,8,1.07,23,43.48,58.33,27.27,L
314,HAVAN,2453,ASCENT,219.4,17.8,14.8,6.0,3.0,138.44,25.2,58.4,12,11,3.87,3.23,1.30,0.53,0.48,14,4,3.0,0.0,2.0,1.0,0.0,0.0,0.0,7,2,21.0,0.91,3.0,0.13,1,0,1,0,0,1,0,0,0,1,0,0,0,1,0,tiob,13,10,8,3,5,7,1.07,23,56.52,72.73,41.67,W
817,VORA,2456,HAVEN,202.2,19.0,17.2,6.4,1.8,127.34,22.2,47.4,14,12,3.65,3.30,1.22,0.54,0.47,23,2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,11,2,25.0,0.96,2.0,0.08,1,1,0,0,0,1,0,0,0,1,0,0,0,1,0,HAVAN,14,12,6,6,6,6,1.07,26,53.85,50.00,50.00,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,DG,22675,BIND,197.8,14.0,16.4,3.4,-2.4,129.52,26.0,50.0,10,11,3.32,3.90,0.82,0.48,0.53,12,3,1.0,0.0,0.0,2.0,0.0,0.0,0.0,4,3,16.0,0.77,2.0,0.10,0,1,0,0,0,0,0,0,1,1,0,0,0,1,1,Rise,8,13,7,5,1,8,0.00,21,38.10,58.33,11.11,L
699,SLK,22683,ICEBOX,232.2,14.4,9.0,3.6,5.4,144.42,22.6,64.6,11,6,4.23,2.65,1.07,0.65,0.36,8,4,4.0,0.0,1.0,0.0,1.0,0.0,0.0,12,1,16.0,0.95,2.0,0.12,0,0,0,1,0,0,0,1,0,0,1,1,0,1,0,hPw,13,4,3,2,10,2,0.00,17,76.47,60.00,83.33,W
966,hPw,22683,ICEBOX,168.0,9.0,14.4,3.4,-5.4,108.92,26.0,44.6,6,11,2.64,4.22,1.01,0.36,0.65,9,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3,2,10.0,0.59,0.0,0.00,1,0,0,0,0,0,0,1,0,0,1,1,0,1,0,SLK,4,13,2,10,2,3,0.00,17,23.53,16.67,40.00,L
698,SLK,22683,HAVEN,239.0,15.8,9.0,2.8,6.8,144.14,30.6,66.8,15,3,4.39,2.51,0.78,0.85,0.18,13,2,3.0,0.0,1.0,1.0,0.0,0.0,0.0,6,3,18.0,1.00,2.0,0.12,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,hPw,13,5,9,3,4,2,0.00,18,72.22,75.00,66.67,W


In [72]:
base_agg[(base_agg.FKPR > 1)]

,Team_x,match_id,Map,ACS,K,D,A,KD_DIFF,ADR,HS%,ECON,FK,FD,KPR,DPR,APR,FKPR,FDPR,2K,3K,4K,5K,1v1,1v2,1v3,1v4,1v5,PL,DE,total_mult_kill,MKPR,total_clutch,CPR,jett,raze,phoenix,reyna,yoru,omen,brimstone,viper,astra,cypher,sage,killjoy,breach,sova,skye,Opp_Team_x,rounds_won,rounds_lost,ct_rounds_won,ct_rounds_lost,t_rounds_won,t_rounds_lost,total_rounds,win_rate,ct_wr,t_wr,RESULT
939,ex-B,16846,BIND,210.0,18.2,17.2,3.6,1.0,127.90,35.8,47.6,6,18,inf,inf,inf,inf,inf,15,7,1.0,1.0,2.0,1.0,0.0,0.0,0.0,8,2,24.0,inf,3.0,inf,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,paiN,0,0,7,5,6,6,0,0.00,58.33,50.00,L
940,ex-B,16846,SPLIT,215.0,41.6,38.4,10.4,3.2,136.42,30.4,51.4,25,29,8.67,8.00,2.16,1.04,1.21,22,4,0.0,1.0,1.0,0.0,0.0,1.0,0.0,12,1,27.0,1.13,2.0,0.08,1,1,0,0,0,0,0,0,1,1,0,0,0,0,1,paiN,13,11,4,8,8,4,24,54.17,33.33,66.67,W
1050,paiN,16846,BIND,211.4,17.2,18.2,5.0,-1.0,131.84,20.0,50.0,18,6,inf,inf,inf,inf,inf,17,5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,6,3,22.0,inf,4.0,inf,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ex-B,0,0,6,6,5,7,0,0.00,50.00,41.67,L
1051,paiN,16846,SPLIT,210.2,38.4,41.6,12.8,-3.2,134.14,19.2,44.8,29,25,8.00,8.66,2.67,1.20,1.04,21,8,0.0,0.0,7.0,0.0,0.0,0.0,0.0,9,3,29.0,1.21,7.0,0.28,0,1,0,1,0,0,0,0,1,1,0,0,0,1,0,ex-B,11,13,4,8,8,4,24,45.83,33.33,66.67,L


In [71]:
base_geral[base_geral.match_id == 10295]

,Player,Agents,ACS,K,D,A,KD_DIFF,ADR,HS%,FK,FD,FK_FD_DIFF,match_id,Team_x,Opp_Team_x,rounds_won,rounds_lost,ct_rounds_won,ct_rounds_lost,t_rounds_won,t_rounds_lost,Map,total_rounds,win_rate,KPR,DPR,APR,FKPR,FKWR,FDPR,ct_wr,t_wr,RESULT,2K,3K,4K,5K,1v1,1v2,1v3,1v4,1v5,ECON,PL,DE,Team_y,Opp_Team_y,total_mult_kill,total_clutch,0,astra,breach,brimstone,cypher,jett,killjoy,omen,phoenix,raze,reyna,sage,skye,sova,viper,yoru,MKPR,CPR
5173,delevingne,reyna,359,20,3,9,17,160.4,11,2,0,2,10295,Vora,HAVAN,13,2,11,1,2,1,BIND,15,86.67,1.33,0.20,0.60,0.13,100.00,0.00,91.67,66.67,W,5,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,100,0,2,Vora,HAVAN,6.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.40,0.00
5177,dragonite,cypher,306,17,7,3,10,172.9,22,3,2,1,10295,Vora,HAVAN,13,2,11,1,2,1,BIND,15,86.67,1.13,0.47,0.20,0.20,60.00,0.13,91.67,66.67,W,4,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,83,0,0,Vora,HAVAN,6.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.40,0.00
5181,fzkk,raze,265,15,8,3,7,158.3,7,3,2,1,10295,Vora,HAVAN,13,2,11,1,2,1,BIND,15,86.67,1.00,0.53,0.20,0.20,60.00,0.13,91.67,66.67,W,4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,76,0,0,Vora,HAVAN,5.0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.33,0.00
5185,krain,sova,166,10,2,7,8,104.0,26,0,0,0,10295,Vora,HAVAN,13,2,11,1,2,1,BIND,15,86.67,0.67,0.13,0.47,0.00,0.00,0.00,91.67,66.67,W,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60,1,3,Vora,HAVAN,3.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.20,0.00
5189,v1xen,omen,152,8,7,7,1,81.5,10,2,1,1,10295,Vora,HAVAN,13,2,11,1,2,1,BIND,15,86.67,0.53,0.47,0.47,0.13,66.67,0.07,91.67,66.67,W,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32,2,0,Vora,HAVAN,3.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.20,0.00
5193,shion,sova,218,8,15,5,-7,156.3,24,2,0,2,10295,HAVAN,Vora,2,13,1,2,1,11,BIND,15,13.33,0.53,1.00,0.33,0.13,100.00,0.00,33.33,8.33,L,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61,0,0,HAVAN,Vora,2.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.13,0.00
5197,pMdr1,jett,133,7,13,0,-6,142.3,36,1,2,-1,10295,HAVAN,Vora,2,13,1,2,1,11,BIND,15,13.33,0.47,0.87,0.00,0.07,33.33,0.13,33.33,8.33,L,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59,0,0,HAVAN,Vora,1.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.07,0.00
5201,Myssen,cypher,130,5,15,1,-10,80.5,24,1,4,-3,10295,HAVAN,Vora,2,13,1,2,1,11,BIND,15,13.33,0.33,1.00,0.07,0.07,20.00,0.27,33.33,8.33,L,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32,1,0,HAVAN,Vora,1.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.07,0.00
5205,pleets,brimstone,111,6,13,6,-7,63.4,10,1,0,1,10295,HAVAN,Vora,2,13,1,2,1,11,BIND,15,13.33,0.40,0.87,0.40,0.07,100.00,0.00,33.33,8.33,L,1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,27,5,1,HAVAN,Vora,1.0,1.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.07,0.07
5209,liazzi,raze,65,1,15,4,-14,56.9,18,0,4,-4,10295,HAVAN,Vora,2,13,1,2,1,11,BIND,15,13.33,0.07,1.00,0.27,0.00,0.00,0.27,33.33,8.33,L,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24,0,0,HAVAN,Vora,0.0,0.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.00,0.00


In [32]:
base_agg.total_rounds.min()

0